In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
insurance=pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
# One-hot encoding
insurance_one_hot=pd.get_dummies(insurance)
insurance_one_hot.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [ ]:
# Create X & y values(features and labels)
X=insurance_one_hot.drop('charges',axis=1)
y=insurance_one_hot['charges']

In [ ]:
# Create training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
tf.random.set_seed(42)

# 1. Create a model
insurance_model=tf.keras.Sequential([
  # Hidden Layer consisting of 100 neurons                                
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(100),
  # Output layer of 1 neuron
  tf.keras.layers.Dense(1)   
  # N.B: We don't need to specify input_shape as it can be automatically handeled by keras                           
])

# 2.Compile a model
insurance_model.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=["mae"])

# 3. Fit the model
insurance_model.fit(X_train,y_train,epochs=100)

Epoch 1/100
34/34 [==============================] - 1s 2ms/step - loss: 12816.6123 - mae: 12816.6123
Epoch 2/100
34/34 [==============================] - 0s 2ms/step - loss: 8691.3799 - mae: 8691.3799
Epoch 3/100
34/34 [==============================] - 0s 2ms/step - loss: 7416.8496 - mae: 7416.8496
Epoch 4/100
34/34 [==============================] - 0s 2ms/step - loss: 7238.1631 - mae: 7238.1631
Epoch 5/100
34/34 [==============================] - 0s 2ms/step - loss: 7148.7202 - mae: 7148.7202
Epoch 6/100
34/34 [==============================] - 0s 2ms/step - loss: 7032.5791 - mae: 7032.5791
Epoch 7/100
34/34 [==============================] - 0s 2ms/step - loss: 6913.5605 - mae: 6913.5605
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 6769.1440 - mae: 6769.1440
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 6653.7939 - mae: 6653.7939
Epoch 10/100
34/34 [==============================] - 0s 2ms/step - loss: 6602.2593 - mae: 6602.25

In [ ]:
insurance_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 100)               1200      
_________________________________________________________________
dense_61 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_62 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 101       
Total params: 21,501
Trainable params: 21,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Evaluate the model on test data
insurance_model.evaluate(X_test,y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3175.4165 - mae: 3175.4165


[3175.41650390625, 3175.41650390625]